In [1]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json, requests, pickle

In [13]:
# Kakao Appkey 저장 파일
APP_KEY_SAVED_FILE = 'res/kakao_msg/app_key'

# 앱키 저장        
app_key = ''

f = open(APP_KEY_SAVED_FILE, 'wb')
pickle.dump(app_key, f)
f.close()

# 앱키 읽기
f = open(APP_KEY_SAVED_FILE, 'rb')
app_key = pickle.load(f)
f.close()

In [9]:
# Kakao Token 저장 파일
KAKAO_TOKEN_SAVED_FILE = 'res/kakao_msg/kakao_token.json'

# 토큰 읽기
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

In [7]:
# 토큰 저장
def save_tokens(filename, tokens):
    with open(filename, 'w') as fp:
        json.dump(tokens, fp)


# 토큰 갱신
def update_tokens(app_key, filename):
    tokens = load_tokens(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    
    response = requests.post(url, data=data)

    if response.status_code != 200:
        print("Error!")
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업
        now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        backup_filename = filename + '.' + now
        os.rename(filename, backup_filename)
        
        # 업데이트
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/orderTest', methods = ['GET', 'POST'])
def orderTest():
    params = json.loads(request.get_data(), encoding='utf-8')
    
    if len(params) == 0:
        return jsonify('No Parameter')
    
    params_string = ''
    
    values = []
    
    for key in params.keys():
        values.append(params[key])
        
    name = values[0]
    cost = values[1]
    
    print('이름: ', name, ', 가격: ', cost)
    
    tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

    # 카톡 메세지 보내기
    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

    # request parameter
    headers = {
        "Authorization" : "Bearer " + tokens['access_token']
    }
    
    message = '이름: {0}, 가격: {1}'.format(name, cost)

    data = {
        "template_object" : json.dumps({
            "object_type" : "text",
            "text" : message,
            "link" : {
                "web_url" : "http://localhost:8080"
            }
        })
    }

    # 메세지 전송
    response = requests.post(url, headers=headers, data=data)
    print(response.status_code)

    if response.status_code == 401:
        tokens = update_tokens(app_key, KAKAO_TOKEN_SAVED_FILE)

    return jsonify(params_string)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
<ipython-input-14-40bf78481dfe>:6: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  params = json.loads(request.get_data(), encoding='utf-8')
127.0.0.1 - - [05/Oct/2021 19:29:58] "POST /orderTest HTTP/1.1" 200 -


이름:  안녕 , 가격:  200000
200
